In [138]:
import pandas as pd
import altair as alt
from altair import datum
import numpy as np

# Summary of Analysis

## overview of dataset
<ul>
    <li>Data acquired via Detroit's Open Data Portal API between 9/20/16 and 8/25/23 (current date)
        <li>Filtered for 6 mental-health-related call types- MNTLPPRS, MNTLARM,MNTLNARM, WBC, SUICIP, SUICTHRT
        <li>Missing data for July 2017, September 2018, September-October 2019, September-October 2020, though data seems complete from January 2021 onwards
            </ul>

## summary of findings
<ul>
<li>Decrease across all 6 categories between year leading up to 988 (July 2021 to June 2022) and the year after (July 2022 to June 2023), though it's not possible to establish causation
<li>Sharpest increase overall for WBC (Well Being Check) into 2021
    <li>Findings about seasonal differences are not valid, b/c of missing data
</ul>

## possible next steps
<ul>
    <li>Try to figure out why data are missing
        <li>look at 30-day data to see if there's anything on 988 referrals
    </ul>

## Documentation
911 calls from <a href="https://data.detroitmi.gov/datasets/detroitmi::911-calls-for-service/explore">Detroit's open data portal, up to 8/23/23</a><br>
See also <a href="https://docs.google.com/spreadsheets/d/1_ZBQGJFBH8ywIToF5yr3qn6Wieyy6-S9l2wwEoGwJjs/edit#gid=502057208">my csv of 91 data up to December 2022</a>, for manual review of records. 

## notes and lessons learned
<ul>
<li>API-retrieved dataset is different from the prepackaged download site, e.g. not missing winter months
    <li>strings and dates are both stored as "Object" in pandas
        </ul>

# Import Data
I queried directly from ESRI to get most recent data (otherwise had nothing newer than September 2022). See my other notebook in this folder.

In [139]:
df_detroit = pd.read_csv("2023_08_25_detroit_911_mh.csv", usecols = ["incident_id","call_timestamp", "category"])

# Prepare Data for Analysis

In [140]:
#remove extra spaces first, b/c all categories use 8 characters regardless
df_detroit["category"] = df_detroit["category"].str.strip()

### decode the datestamp
incident ID 201626402101 has a timestamp 1474413140000<br>
looking up the actual record, it should be 2016-09-20 19:12:20-04:00<br>
per https://www.unixtimestamp.com/, this is a UNIX timestamp, seconds since January 1, 1970. But I get an "“Out of bounds nanosecond timestamp" error interpreting this as seconds, and so had to define as milliseconds

In [141]:
df_detroit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103267 entries, 0 to 103266
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   incident_id     103267 non-null  int64 
 1   category        103267 non-null  object
 2   call_timestamp  103267 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 2.4+ MB


In [142]:
df_detroit["call_date"]= pd.to_datetime(df_detroit['call_timestamp'], unit='ms')

### create month and year fields

In [143]:
#hyphen before m removes leading zero, but I want to keep it
df_detroit['month']=df_detroit['call_date'].dt.strftime('%m').astype(int)
df_detroit['year']=df_detroit['call_date'].dt.strftime('%Y').astype(int)
#df_detroit['call_quarter']=(df_detroit['call_month'].astype(int)-1)/3
#df_detroit['call_month_year']=df_detroit['call_year'].astype(str) + "-" + df_detroit['call_month'].astype(str)

In [144]:
df_detroit.head()

,incident_id,category,call_timestamp,call_date,month,year
0,201626402101,MENTPPRS,1474413140000,2016-09-20 23:12:20,9,2016
1,201626402108,WBC,1474413282000,2016-09-20 23:14:42,9,2016
2,201626401001,MENTPPRS,1474391088000,2016-09-20 17:04:48,9,2016
3,201626401066,MENTPPRS,1474392442000,2016-09-20 17:27:22,9,2016
4,201626401162,MENTPPRS,1474394806000,2016-09-20 18:06:46,9,2016


### Look at date range and create a dataset for full years

In [145]:
#get the date range for the dataset
df_detroit.call_date.min()

Timestamp('2016-09-20 12:08:35')

In [146]:
#get the date range for the dataset
df_detroit.call_date.max()

Timestamp('2023-08-25 00:35:38')

In [147]:
df_2017_2022 = df_detroit[(df_detroit['year'] >=2017) & (df_detroit['year'] <=2022)]

### look for gaps in the data- Yikes! missing data for July 2017, September 2018, September-October 2019, September-October 2020

In [148]:
df_all_months = df_detroit.groupby(['year','month']).month.value_counts().reset_index()
df_months_sum = df_all_months.pivot_table(index="year",columns="month")
df_months_sum

count                                                                  \
month      1       2       3       4       5       6       7       8       9    
year                                                                            
2016      NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   360.0   
2017   1172.0  1028.0  1150.0  1079.0  1336.0   127.0     NaN   514.0  1169.0   
2018   1296.0  1125.0  1307.0  1229.0  1386.0  1318.0  1342.0   532.0     NaN   
2019   1317.0  1180.0  1290.0  1278.0  1335.0  1374.0  1448.0   661.0     NaN   
2020   1385.0  1270.0  1406.0  1306.0  1469.0  1471.0  1512.0  1533.0     NaN   
2021   1450.0  1278.0  1561.0  1541.0  1696.0  1678.0  1633.0  1783.0  1546.0   
2022   1655.0  1540.0  1741.0  1708.0  1954.0  1900.0  1765.0  1895.0  1770.0   
2023   1282.0  1321.0  1518.0  1422.0  1564.0  1536.0  1559.0  1126.0     NaN   

                               
month      10      11      12  
year                           
2016   1123.0  1111.0  1138.0  
2017   1164.0  1142.0  1203.0  
2018   1128.0  1222.0  1284.0  
2019      NaN   612.0  1249.0  
2020      NaN  1377.0  1500.0  
2021   1583.0  1539.0  1540.0  
2022    455.0  1379.0  1391.0  
2023      NaN     NaN     NaN

### Look at data by category

In [43]:
category_dict = {
    'WBC': 'Well Being Check',
    'MNTLNARM': 'Mental Violent Not Armed',
    'MENTPPRS': 'Mental Not Violent',
    'SUICTHRT': 'Suicide Threat',
    'SUICIP': 'Suicide In Progress',
    'MNTLARM': 'Mental Violent Armed'
}

In [44]:
df_detroit['category_desc'] = df_detroit['category'].map(category_dict)
df_2017_2022['category_desc'] = df_2017_2022['category'].map(category_dict)

/tmp/ipykernel_2172/4236746070.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2017_2022['category_desc'] = df_2017_2022['category'].map(category_dict)


In [45]:
#all_years
df_detroit["category_desc"].value_counts()

category_desc
Well Being Check            30836
Mental Violent Not Armed    27298
Mental Not Violent          15602
Suicide Threat              13482
Suicide In Progress          8375
Mental Violent Armed         7567
Name: count, dtype: int64

In [46]:
#full years only
df_2017_2022["category_desc"].value_counts()

category_desc
Well Being Check            26299
Mental Violent Not Armed    23372
Mental Not Violent          13214
Suicide Threat              11730
Suicide In Progress          7171
Mental Violent Armed         6421
Name: count, dtype: int64

# Summarize By Year
To look for overall trends, but look at only whole years

In [47]:
df_by_year = df_2017_2022.groupby(['year']).category.value_counts().reset_index()
df_by_year.head()

,year,category,count
0,2017,MNTLNARM,3273
1,2017,WBC,2512
2,2017,MENTPPRS,1803
3,2017,SUICTHRT,1369
4,2017,SUICIP,1175


In [48]:
df_by_year_chart = df_by_year.copy()
df_by_year_chart['category'] = df_by_year_chart['category'].map(category_dict)

In [49]:
alt.Chart(df_by_year_chart).mark_line().encode(
    x='year',
    y=alt.Y('count',scale=alt.Scale(domain=[0,7000])),
    color ='category'
).configure_axisX(
    tickCount=len(df_by_year_chart['year'].unique()), # Number of unique years
    format=".0f"  # Remove commas from the labels
)

alt.Chart(...)

### Look at Percent of Calls by Category
since numbers look up across the board...

In [50]:
df_pivot_year = df_by_year.pivot_table(index="year",columns="category")
#get rid of multilevel index, rename unnamed pivot table index
df_pivot_year = df_pivot_year.droplevel(0,axis=1)
df_pivot_year = df_pivot_year.rename(columns={'':'year'})
df_pivot_year = df_pivot_year.reset_index()
df_pivot_year

category,year,MENTPPRS,MNTLARM,MNTLNARM,SUICIP,SUICTHRT,WBC
0,2017,1803,952,3273,1175,1369,2512
1,2018,2010,1179,3770,1191,1914,3105
2,2019,1716,817,3245,1059,1770,3137
3,2020,2053,972,3589,1166,1783,4666
4,2021,2794,1197,4658,1339,2349,6491
5,2022,2838,1304,4837,1241,2545,6388


In [51]:
df_pivot_year["total"]= df_pivot_year["MENTPPRS"]+df_pivot_year["MNTLARM"]\
+df_pivot_year["MNTLNARM"]+df_pivot_year["SUICIP"]+df_pivot_year["SUICTHRT"]+df_pivot_year["WBC"]
df_pivot_year

category,year,MENTPPRS,MNTLARM,MNTLNARM,SUICIP,SUICTHRT,WBC,total
0,2017,1803,952,3273,1175,1369,2512,11084
1,2018,2010,1179,3770,1191,1914,3105,13169
2,2019,1716,817,3245,1059,1770,3137,11744
3,2020,2053,972,3589,1166,1783,4666,14229
4,2021,2794,1197,4658,1339,2349,6491,18828
5,2022,2838,1304,4837,1241,2545,6388,19153


In [52]:
df_pivot_year["pMENTPPRS"] = df_pivot_year["MENTPPRS"] / df_pivot_year["total"]
df_pivot_year["pMNTLARM"] = df_pivot_year["MNTLARM"]/df_pivot_year["total"]
df_pivot_year["pMNTLNARM"] = df_pivot_year["MNTLNARM"]/df_pivot_year["total"]
df_pivot_year["pSUICIP"] = df_pivot_year["SUICIP"]/df_pivot_year["total"]
df_pivot_year["pSUICTHRT"] = df_pivot_year["SUICTHRT"]/df_pivot_year["total"]
df_pivot_year["pWBC"] = df_pivot_year["WBC"]/df_pivot_year["total"]
df_pivot_year = df_pivot_year.drop(["MENTPPRS","MNTLARM","MNTLNARM","SUICIP","SUICTHRT","WBC"],axis=1)
# df_pivot_year = df_pivot_year.rename(columns={'pct17':2017,'pct18':2018,'pct19':2019,'pct20':2020,'pct21':2021,'pct22':2022})
df_pivot_year

category,year,total,pMENTPPRS,pMNTLARM,pMNTLNARM,pSUICIP,pSUICTHRT,pWBC
0,2017,11084,0.162667,0.085890,0.295291,0.106009,0.123511,0.226633
1,2018,13169,0.152631,0.089528,0.286278,0.090440,0.145341,0.235781
2,2019,11744,0.146117,0.069567,0.276311,0.090174,0.150715,0.267115
3,2020,14229,0.144283,0.068311,0.252231,0.081945,0.125307,0.327922
4,2021,18828,0.148396,0.063576,0.247397,0.071117,0.124761,0.344752
5,2022,19153,0.148175,0.068083,0.252545,0.064794,0.132877,0.333525


In [53]:
df_mh_by_year = pd.melt(df_pivot_year, id_vars=['year'])
df_mh_by_year = df_mh_by_year.drop(df_mh_by_year.query("category=='total'").index)
#df_mh_by_year.reset_index(drop=True)
#df_mh_by_year['category']= df_mh_by_year['category'].astype(str)
df_mh_by_year.reset_index(drop=True, inplace=True)
df_mh_by_year

,year,category,value
0,2017,pMENTPPRS,0.162667
1,2018,pMENTPPRS,0.152631
2,2019,pMENTPPRS,0.146117
3,2020,pMENTPPRS,0.144283
4,2021,pMENTPPRS,0.148396
5,2022,pMENTPPRS,0.148175
6,2017,pMNTLARM,0.085890
7,2018,pMNTLARM,0.089528
8,2019,pMNTLARM,0.069567
9,2020,pMNTLARM,0.068311


In [54]:
alt.Chart(df_mh_by_year).mark_line().encode(
    x='year',
    y=alt.Y('value',scale=alt.Scale(domain=[0,1])),
    color ='category'
).configure_axisX(
    tickCount=len(df_mh_by_year['year'].unique()), # Number of unique years
    format=".0f"  # Remove commas from the labels
)

alt.Chart(...)

### Summarize By Year, July to June

In [63]:
year_dict = {
    2017: '2017-18',
    2018: '2018-19',
    2019: '2019-20',
    2020: '2020-21',
    2021: '2021-22',
    2022: '2022-23',
}

In [64]:
df_2017j_2022j= df_detroit.filter(['category','month','year','category_desc'],axis=1)
df_2017j_2022j["half"]= np.where(df_2017j_2022j.month >=7,2,1)
df_2017j_2022j['jYear']=np.where(df_2017j_2022j.half ==2,df_2017j_2022j['year'],df_2017j_2022j['year']-1)
#df_2017j_2022j["jMonth"]=np.where(df_2017j_2022j.half ==2,df_2017j_2022j['month']-6,df_2017j_2022j['month']+6)
df_2017j_2022j = df_2017j_2022j[(df_2017j_2022j['jYear'] >=2017) & (df_2017j_2022j['jYear'] <=2022)]
df_2017j_2022j['lblYear'] = df_2017j_2022j['jYear'].map(year_dict)
df_2017j_2022j

,category,month,year,category_desc,half,jYear,lblYear
9604,SUICTHRT,8,2017,Suicide Threat,2,2017,2017-18
9605,SUICIP,8,2017,Suicide In Progress,2,2017,2017-18
9606,MNTLARM,8,2017,Mental Violent Armed,2,2017,2017-18
9607,SUICIP,8,2017,Suicide In Progress,2,2017,2017-18
9608,MENTPPRS,8,2017,Mental Not Violent,2,2017,2017-18
...,...,...,...,...,...,...,...
100577,MNTLARM,6,2023,Mental Violent Armed,1,2022,2022-23
100578,SUICTHRT,6,2023,Suicide Threat,1,2022,2022-23
100579,SUICTHRT,6,2023,Suicide Threat,1,2022,2022-23
100580,WBC,6,2023,Well Being Check,1,2022,2022-23


In [77]:
df_by_jyear_chart = df_2017j_2022j.groupby(['jYear','lblYear']).category_desc.value_counts().reset_index()
df_by_jyear_chart.head()

,jYear,lblYear,category_desc,count
0,2017,2017-18,Mental Violent Not Armed,3673
1,2017,2017-18,Well Being Check,2979
2,2017,2017-18,Mental Not Violent,2064
3,2017,2017-18,Suicide Threat,1728
4,2017,2017-18,Suicide In Progress,1240


In [84]:
alt.Chart(df_by_jyear_chart).mark_line().encode(
    x='jYear',
    y=alt.Y('count',scale=alt.Scale(domain=[0,7000])),
    color ='category_desc',
).configure_axisX(
    tickCount=len(df_by_jyear_chart['jYear'].unique()), # Number of unique years
    format=".0f"  # Remove commas from the labels
)
# alt.Chart(df_by_jyear_chart).mark_line().transform_calculate(
#   x_label="datum.lblYear"  
# ).encode(
#     x=alt.X('jYear:N', title='Year'),
#     y=alt.Y('count',scale=alt.Scale(domain=[0,7000])),
#     color ='category_desc',
#     text=alt.Text('x_label:N')
# ).configure_axisX(
#     tickCount=len(df_by_jyear_chart['jYear'].unique()), # Number of unique years
#     format=".0f"  # Remove commas from the labels
# )

alt.Chart(...)

# Summarize By Month to Assess Seasonality
<div style="color:red;">This analysis is flawed due to missing data</div>

In [ ]:
df_by_month = df_2017_2022.groupby(['month']).category.value_counts().reset_index()
df_by_month["category_desc"]=df_by_month['category'].map(rename_dict)

In [ ]:
alt.Chart(df_by_month).mark_line().encode(
    x='month',
    y=alt.Y('count',scale=alt.Scale(domain=[0,4000])),
    color ='category_desc'
).configure_axisX(
    tickCount=len(df_by_month['month'].unique()) # Number of unique years
)

# Compare pre-988 (August 2021-July 2022) to post-988 (August 2022-July 2023)

In [97]:
df_post988 = df_detroit.groupby(['month','year']).category.value_counts().reset_index()
df_post988

,month,year,category,count
0,1,2017,MNTLNARM,366
1,1,2017,WBC,244
2,1,2017,MENTPPRS,205
3,1,2017,SUICTHRT,132
4,1,2017,SUICIP,132
...,...,...,...,...
463,12,2022,MNTLNARM,353
464,12,2022,MENTPPRS,201
465,12,2022,SUICTHRT,183
466,12,2022,MNTLARM,107


In [107]:
df_post988_cats = df_post988.groupby(['year']).category.value_counts().reset_index()
df_post988_cats

,year,category,count
0,2016,MNTLARM,4
1,2016,WBC,4
2,2016,SUICTHRT,4
3,2016,SUICIP,4
4,2016,MNTLNARM,4
5,2016,MENTPPRS,4
6,2017,WBC,11
7,2017,SUICTHRT,11
8,2017,SUICIP,11
9,2017,MNTLNARM,11


In [104]:
df_post988_cats.describe()

,year,count
count,8.00000,8.000000
mean,2019.50000,58.500000
std,2.44949,15.955295
min,2016.00000,24.000000
25%,2017.75000,57.000000
50%,2019.50000,63.000000
75%,2021.25000,67.500000
max,2023.00000,72.000000


In [ ]:
df_post988["month_index"]=12*df_post988["year"] + df_post988["month"]
df_post988["nYear"]=round((df_post988["month_index"]-(2022*12+7))/12,0)
df_post988["nYear"] = df_post988["nYear"].astype(int)
df_post988["nMonth"]=(df_post988["month_index"]-(2022*12+7))%12
df_post988 = df_post988.query("nYear>=0")
df_post988["seriesYear"]= np.where(df_post988["nYear"]==0,"pre-988","post-988")
df_post988

In [ ]:
# condition1 = ((df_by_month["year"] == 2021) & (df_by_month["month"] >= 8))
# condition2 = ((df_by_month["year"] == 2022) & (df_by_month["month"] <= 7))
# df_by_month["2021-22"]=np.where(condition1 | condition2,1,0)
# condition3 = ((df_by_month["year"] == 2022) & (df_by_month["month"] >= 8))
# condition4 = ((df_by_month["year"] == 2023) & (df_by_month["month"] <= 7))
# df_by_month["2022-23"]=np.where(condition3 | condition4,2,0)
# df_by_month["nYear"]= df_by_month["2021-22"]+df_by_month["2022-23"]
# df_by_month

In [ ]:
alt.Chart(df_post988).mark_line().encode(
    x='nMonth',
    y=alt.Y('count',scale=alt.Scale(domain=[0,1000])),
    color ='seriesYear'
).configure_axisX(
    tickCount=len(df_post988['nMonth'].unique()) # Number of unique years
).transform_filter(
    (datum.category == "WBC"))

In [ ]:
alt.Chart(df_post988).mark_line().encode(
    x='nMonth',
    y=alt.Y('count',scale=alt.Scale(domain=[0,1000])),
    color ='seriesYear'
).configure_axisX(
    tickCount=len(df_post988['nMonth'].unique()) # Number of unique years
).transform_filter(
    (datum.category == "MENTPPRS"))